<a href="https://colab.research.google.com/github/tomasonjo/mcp-workshop/blob/main/01-mcp-cypher.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install --quiet --upgrade fastmcp langchain-mcp-adapters langgraph langchain-openai mcp-neo4j-cypher

In [2]:
import threading
import time
import logging
import sys
from io import StringIO
from fastmcp import FastMCP, Client
from fastmcp.client.transports import StdioTransport

# Set root logger to CRITICAL
logging.getLogger().setLevel(logging.CRITICAL)

# Suppress all MCP/uvicorn/FastAPI related loggers
loggers_to_suppress = [
    "uvicorn",
    "uvicorn.access",
    "uvicorn.error",
    "fastmcp"
]

for logger_name in loggers_to_suppress:
    logger = logging.getLogger(logger_name)
    logger.setLevel(logging.CRITICAL)
    logger.propagate = False

# Also disable the default stream handler
logging.disable(logging.WARNING)

/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


In [3]:
import os
import getpass

os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter your OpenAI API key")

Enter your OpenAI API key··········


In [4]:
from langchain_openai import ChatOpenAI

model_name = 'gpt-5-mini'
llm = ChatOpenAI(model=model_name)

/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


In [5]:
# Run this if you are using a local environment
"""
# Create main server
mcp = FastMCP("Movies Server")

# Create stdio transport for Neo4j MCP server
transport = StdioTransport(
    command="uvx",
    args=["mcp-neo4j-cypher@0.5.1", "--namespace", "movies"],
    env={
        "NEO4J_URI": "neo4j+s://demo.neo4jlabs.com",
        "NEO4J_USERNAME": "recommendations",
        "NEO4J_PASSWORD": "recommendations",
        "NEO4J_DATABASE": "recommendations"
    }
)

# Create client with the transport
backend = Client(transport)

# Create and mount proxy (not async)
proxy = FastMCP.as_proxy(backend, name="Neo4jProxy")
mcp.mount(proxy, prefix="movies")

# Run server silently in background thread
def run_server():
    mcp.run(transport="streamable-http", port=8000)

server_thread = threading.Thread(target=run_server, daemon=True)
server_thread.start()
time.sleep(2)

print("✓ Server ready with Neo4j tools mounted")
print("Server running at: http://localhost:8000")
"""

'\n# Create main server\nmcp = FastMCP("Movies Server")\n\n# Create stdio transport for Neo4j MCP server\ntransport = StdioTransport(\n    command="uvx",\n    args=["mcp-neo4j-cypher@0.5.1", "--namespace", "movies"],\n    env={\n        "NEO4J_URI": "neo4j+s://demo.neo4jlabs.com",\n        "NEO4J_USERNAME": "recommendations",\n        "NEO4J_PASSWORD": "recommendations",\n        "NEO4J_DATABASE": "recommendations"\n    }\n)\n\n# Create client with the transport\nbackend = Client(transport)\n\n# Create and mount proxy (not async)\nproxy = FastMCP.as_proxy(backend, name="Neo4jProxy")\nmcp.mount(proxy, prefix="movies")\n\n# Run server silently in background thread\ndef run_server():\n    mcp.run(transport="streamable-http", port=8000)\n\nserver_thread = threading.Thread(target=run_server, daemon=True)\nserver_thread.start()\ntime.sleep(2)\n\nprint("✓ Server ready with Neo4j tools mounted")\nprint("Server running at: http://localhost:8000")\n'

In [6]:
# Run this if running in Google Colab
import threading
import subprocess
import time

def run_server():
    subprocess.run([
        "mcp-neo4j-cypher",
        "--server-port", "8000",
        "--db-url", "neo4j+s://demo.neo4jlabs.com",
        "--username", "recommendations",
        "--password", "recommendations",
        "--database", "recommendations",
        "--transport", "http"
    ])

server_thread = threading.Thread(target=run_server, daemon=True)
server_thread.start()
time.sleep(5)

/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


In [7]:
from langchain_mcp_adapters.client import MultiServerMCPClient
from langgraph.prebuilt import create_react_agent

async def query(q: str):
    client = MultiServerMCPClient({
        "neo4j-database": {
            "url": "http://localhost:8000/mcp",
            "transport": "streamable_http"
        }
    })

    tools = await client.get_tools()
    agent = create_react_agent(llm, tools)
    response = await agent.ainvoke({"messages": [{"role": "system", "content": """You are an expert researcher.
Your job is to conduct thorough research, and then write a polished report in markdown. Neo4j database contains all information about movies.
Never ask follow-up questions regarding the style of response. Be concise.
You cannot do CSV export or visualizations!"""},
                                                 {"role": "user", "content": q}]})
    for message in response["messages"]:
        message.pretty_print()

/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


In [8]:
await query("Which actor has played in the most movies after 2010?")

/tmp/ipython-input-841050986.py:13: LangGraphDeprecatedSinceV10: create_react_agent has been moved to `langchain.agents`. Please update your import to `from langchain.agents import create_agent`. Deprecated in LangGraph V1.0 to be removed in V2.0.
  agent = create_react_agent(llm, tools)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


================================ System Message ================================

You are an expert researcher.
Your job is to conduct thorough research, and then write a polished report in markdown. Neo4j database contains all information about movies.
Never ask follow-up questions regarding the style of response. Be concise.
You cannot do CSV export or visualizations!
================================ Human Message =================================

Which actor has played in the most movies after 2010?
================================== Ai Message ==================================
Tool Calls:
  get_neo4j_schema (call_BtUyk8ohxyx5oCfzMcRTI1Xm)
 Call ID: call_BtUyk8ohxyx5oCfzMcRTI1Xm
  Args:
    sample_size: 1000
================================= Tool Message =================================
Name: get_neo4j_schema

{"User": {"type": "node", "count": 671, "properties": {"name": {"indexed": true, "type": "STRING"}, "userId": {"indexed": true, "type": "STRING"}}, "relationships": {"RATED

In [9]:
await query("Find movies where both aliens and kids appear?")

/tmp/ipython-input-841050986.py:13: LangGraphDeprecatedSinceV10: create_react_agent has been moved to `langchain.agents`. Please update your import to `from langchain.agents import create_agent`. Deprecated in LangGraph V1.0 to be removed in V2.0.
  agent = create_react_agent(llm, tools)


================================ System Message ================================

You are an expert researcher.
Your job is to conduct thorough research, and then write a polished report in markdown. Neo4j database contains all information about movies.
Never ask follow-up questions regarding the style of response. Be concise.
You cannot do CSV export or visualizations!
================================ Human Message =================================

Find movies where both aliens and kids appear?
================================== Ai Message ==================================
Tool Calls:
  get_neo4j_schema (call_Hf9DrVgS1NAdCNefA0fmWDP2)
 Call ID: call_Hf9DrVgS1NAdCNefA0fmWDP2
  Args:
    sample_size: 1000
================================= Tool Message =================================
Name: get_neo4j_schema

{"User": {"type": "node", "count": 671, "properties": {"name": {"indexed": true, "type": "STRING"}, "userId": {"indexed": true, "type": "STRING"}}, "relationships": {"RATED": {"di

In [10]:
await query("Create a profitability report for movies per genre per decade.")

/tmp/ipython-input-841050986.py:13: LangGraphDeprecatedSinceV10: create_react_agent has been moved to `langchain.agents`. Please update your import to `from langchain.agents import create_agent`. Deprecated in LangGraph V1.0 to be removed in V2.0.
  agent = create_react_agent(llm, tools)


================================ System Message ================================

You are an expert researcher.
Your job is to conduct thorough research, and then write a polished report in markdown. Neo4j database contains all information about movies.
Never ask follow-up questions regarding the style of response. Be concise.
You cannot do CSV export or visualizations!
================================ Human Message =================================

Create a profitability report for movies per genre per decade.
================================== Ai Message ==================================
Tool Calls:
  get_neo4j_schema (call_VAWeoBEQkvP35onRmdSo2Nbn)
 Call ID: call_VAWeoBEQkvP35onRmdSo2Nbn
  Args:
    sample_size: 1000
================================= Tool Message =================================
Name: get_neo4j_schema

{"User": {"type": "node", "count": 671, "properties": {"name": {"indexed": true, "type": "STRING"}, "userId": {"indexed": true, "type": "STRING"}}, "relationships"